<a href="https://colab.research.google.com/github/jsaraujo5081/03MAIR-Algoritmos-de-optimizacion/blob/master/SEMINARIO/Seminario_Juan_Araujo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Algoritmos de optimización - Seminario<br>
Nombre y Apellidos:  Juan Araujo Dueñas <br>
Url: https://github.com/jsaraujo5081/03MAIR-Algoritmos-de-optimizacion/tree/master/SEMINARIO <br>





                                        

In [0]:
#CARGA DE LIBRERIAS
###################

import pandas as pd
import numpy as np
import random
import scipy.stats as sts
from itertools import combinations

## Problema:

__1. Eleccion de grupos de poblacion homogeneos__



-   Una productora ganadera nos encarga la tarea de seleccionar grupos de
terneros para aplicar 3 tratamientos diferentes. Para cada uno de los
tratamientos debemos seleccionar 3 grupos de terneros que sean lo mas
homogéneos posible en peso para que en los resultados del tratamiento influya
lo menos posible el peso del animal. Disponemos de una población de N
animales entre machos y hembras.
- Aun se desconocen los datos concretos de los animales por lo que debemos
trabajar con datos que debemos generar de forma aleatoria. Lo que si nos
proporcionan es una plantilla de los datos:

|ID|Peso (kg)|Sexo|
|----|--------|----|
|1|38.7|M|
|2|37.1|H|
|3|39.2|M|

-  Hemos podido averiguar que el peso medio del ternero es 37kg con una
desviación estándar de 2.1


- Se solicita diseñar un algoritmo para conseguir una agrupación que cumpla de la mejor manera posible las especificaciones de la productora.


__(\*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones?__ <br>

Respuesta:

Sea $G$ el número total de grupos que se deben construir, y $N$ el número de terneros que tenemos en nuestra población, tal que $G \leq N$. Además de esto sea $T$ el numero de tratamientos tal que $G$ tiene como divisor a $T$.

Sin restricciones en el problema, para conocer el numero total de posibles soluciones ($X$) tendría como base el problema matematico de establecer el numero total posibles particiones de $G$ elementos en un conjunto de numerable y finito de cardinalidad $N$.

Tomemos como ejemplo el caso de un conjunto  $[A,B]$ y determinemos cuantas posibles particiones de 2 elementos pueden realizarse. El resultado es trivial ya que existe solamente una posible particion definida como sigue:

$$[[A],[B]]$$

Ampliemos a un conjunto con $N=3$ definido por $[A,B,C]$. El numero total de particiones con 2 elementos se incrementa a 3:

$$[[A],[B,C]]$$
$$[[B],[A,C]]$$
$$[[C],[A,B]]$$

Si tomamos como otro ejemplo el conjunto $[A,B,C,D]$ y queremos saber cuantas particiones de 2 elementos se pueden hacer el resultado se incrementa a 7:

$$[[A],[B,C,D]]$$
$$[[B],[A,C,D]]$$
$$[[C],[A,B,D]]$$
$$[[D],[A,B,C]]$$
$$[[A,B],[C,D]]$$
$$[[A,C],[B,D]]$$
$$[[A,D],[B,C]]$$

En el siguiente cuadro se resumen las posibles soluciones para los primeros numeros $N \leq 6$ y $G \leq 6$ 

N,G|1|2|3|4|5|6|...
--|--|--|--|--|--|--|--
__1__|1||||||
__2__|1|1|||||
__3__|1|3|1||||
__4__|1|7|6|1|||
__5__|1|15|25|10|1||
__6__|1|31|90|65|15|1|
__...__|...|...|...|...|...|...|...

Las particiones posibles se pueden entonces obtener mediante el numero de Stirling de segundo orden definido por la expresion

$$S(N,G)={\frac {1}{G!}} \sum^G_{i=0}{(-1)^{i}{G \choose i}(G-i)^{N}}$$

Esta ecuacion igualmente presenta una forma recursiva mas sencilla dada por

$$S(N,G)=G \times S(N-1,G) + S(N-1,G-1) $$

con condiciones iniciales $S(0,0)=1$ y $S(N,0)=S(0,G)=0$

Ahora bien, con las particiones posibles, debemos asignarlas en cada uno de los $T$ tratamientos, por lo que las soluciones posibles serían:

$$X = T {G \choose {G/T}} S(N,G)$$

Reemplazando los términos de la expresion por los valores dados en el enunciado se obtiene

$$X = 3 {9 \choose 3} S(N,9)=252 \times S(N,9)$$

<br>
__¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones?:__ <br>

Respuesta:

La restriccion del problema se refiere a que los grupos sean homogeneos tanto en peso (intra e inter grupos) como en numero de individuos. El peso es una condicion endogena de la poblacion de terneros que se tengan y no puede ser evaluada *a priori*,  pero respecto al numero de individuos es posible ver que las particiones que cumplen este requisito se limitan a todas aquellas cuyo conteo de terminos por elemento venga dado $\lfloor N/G \rfloor$. 

Visto esto consideremos el caso sencillo de $N=20$ y el ya establecido $G=9$ que permite obtener un total de 95 particiones posibles con un tamaño de elementos igual a 2 en siete de ellos y 3 en los dos restantes. 

Para formalizar este resultado, podemos afirmar que 

$$X = 252 \times {\frac{1}{2} }{N \choose {\lfloor N/9 \rfloor}} = 126 \times {N \choose {\lfloor N/9 \rfloor}}$$

Una ultima restriccion que se considerara de forma arbitraria en este caso en particular es que se predefinira a $N=200$, por lo que

$$X = 126 \times {200 \choose 22} \approx 1.42 \times 10^{31} $$

__Modelo para el espacio de soluciones<br>
(\*) ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguméntalo)__<br>

Respuesta:

Podemos etiquetar a cada ternero con un identificador secuencial que empieze en 1 y termine en $N$. Esta información quedaría guardada en una lista.

Así también, para la definición del peso es posible crear una lista de $N$ números aleatorios quer sigan la distribución expuesta en el enunciado, tal que 

$$Peso \sim  Normal(\mu = 37, \sigma = 2.1)$$

En lo que respecta al sexo de los terneros, podemos crear un vector aleatorio con $N$ números reales entre 0 y 1. A continuación, en una lista asignamos "H" si el i-esimo numero aleatorio generado es mayor o igual a 0.5, caso contrario asignamos "M".  

Finalmente, estos tres vectores los podemos guardar en un diccionario que identifique por su nombre a cada uno. 

Con esto, se facilita la construcción de un data frame conforme la plantilla del enunciado y su procesamiento en los algoritmos a mostrar en líneas siguientes.

De igual forma, se facilita la ubicación de observaciones específicas al momento de evaluar diferentes atributos como el Peso o el Sexo utilizando búsqueda posicional sobre la base de los índices del data frame.

In [0]:
#GENERACION DE ESTRUCTURA DE DATOS
##################################

#Parametros definidos inicialmente
#---------------------------------
mu = 37
sigma = 2.1
N = 200 #Esto es un parametro predefinido por el autor a modo de restriccion

#Lista de ID
#-----------
ID = [i+1 for i in range(N)]
#print(ID)

#Lista de pesos
#--------------
peso = np.random.normal(mu,sigma,N)
#print(peso)

#Lista de sexos
#--------------
sexo = []
mask = [random.uniform(0, 1) for i in range(N)]
for i in range(N):
  if mask[i] < 0.5: 
    sexo.append("M")
  else:
    sexo.append("H")
#print(sexo)
    
#Dicionario de listas para posterior construccion de dataframe
#-------------------------------------------------------------
dicc = {"ID":ID,
        "Peso":peso,
        "Sexo":sexo}
#print(dicc)

__Según el modelo para el espacio de soluciones__<br>
__(\*)¿Cual es la función objetivo?__<br>

Respuesta:

La función objetivo en este caso corresponde a aquella que me permita evaluar la homogeneidad de los grupos que vayamos creando tanto de forma interna como externa con la variable del peso de los terneros. 

En este sentido, para la intra - homogeneidad se buscara alcanzar el valor minimo del promedio de las desviaciones estandar obtenidas en los grupos, mientras que para la inter - homogeneidad se buscara el valor minimo de la desviacion estandar de los promedios de los grupos  

<br>
__(\*)¿Es un problema de maximización o minimización?__ <br>
Respuesta:

Vista esa función objetivo que arroja dos outputs el problema es de minimización.

In [0]:
#GENERACION DE FUNCION OBJETIVO
###############################

#grupos: data sets en el se está evaluando la homogeneidad
#x: número de data sets
def func_objetivo(grupos,x):
  
  #Calculo de la desviacion estandar
  std_grupos = [np.std(grupos[i]["Peso"]) for i in range(x)]
  std_promedio = np.average(std_grupos)
  
  #Calculo del promedio
  prom_grupos = [np.mean(grupos[i]["Peso"]) for i in range(x)]
  prom_desviacion = np.std(prom_grupos)
  
  return std_promedio,prom_desviacion

__Diseña un algoritmo para resolver el problema por fuerza bruta__<br>
Respuesta:

Por fuerza bruta, implica que debemos recorrer todo el universo de posibles soluciones hasta encontrar aquella partición donde exista la mayor homogeneidad, sobre la base de su desviación estándar y el promedio como función objetivo multivariable.

Con este proposito, en una primera instancia se contruye una funcion auxiliar que permite generar las particiones posibles y se la aplica en el algoritmo por fuerza bruta para generar los grupos totales deseados.

A continuacion se construye una funcion que genere todas las posibles combinaciones de estos grupos para asignarlos a los tratamientos evaluando nuevamente la homogeneidad, y se determina la mejor asignacion en los tratamientos.

In [0]:
##FUNCIONES AUXILIARES PARA GENERAR TODAS LAS PARTICIONES
#########################################################

#Generar todas las particiones
#-----------------------------
#Para esta función recursiva se ha tomado como referencia a: https://stackoverflow.com/questions/19368375/set-partitions-in-python
def particiones_all(S):
    if len(S) == 1:
        yield [S]
        return
    first = S[0]
    for smaller in particiones_all(S[1:]):
        for n, subset in enumerate(smaller):
            yield smaller[:n] + [[ first ] + subset]  + smaller[n+1:] 
        yield [[ first ]] + smaller
        
#Obtener solamente las particiones de tamaño x y homogeneas en tamaño de elementos
#---------------------------------------------------------------------------------
def particiones_x(S,x):
  particiones = particiones_all(S)
  part_final = []
  tam_elem = len(S)//x
  for p in particiones:
    size = []
    for elem in p:
      size.append(len(elem)) 
    if (len(p) == x) and (min(size) == tam_elem):
      part_final.append(p)
  return part_final

In [0]:
#ALGORITMO POR FUERZA BRUTA PARA GENERAR GRUPOS
###############################################

#df_terneros: dataframe con los datos a analizar
#x: numero de grupos a crear
def agrupamiento_fuerzabruta(df_terneros,x):
  
  #Inicializaciones globales 
  MIN_Std = 10e10 
  MIN_Prom = 10e10
  MEJORES_GRUPOS = []
  
  #Generacion de particiones
  part_x = particiones_x(list(df_terneros["ID"]),x)
  
  #Evaluacion e funcion objetivo en cada particion
  for particion in part_x:
    
    df_part = []
    
    for elem in particion:
      df_tmp = df_terneros[df_terneros["ID"].isin(elem)]
      df_part.append(df_tmp)
  
    CASO_Std, CASO_Prom = func_objetivo(df_part,x) 
  
    if (CASO_Std < MIN_Std) and (CASO_Prom < MIN_Prom):
      MIN_Std = CASO_Std
      MIN_Prom = CASO_Prom
      MEJORES_GRUPOS = df_part
  
  #Resultados obtenidos
  print("Promedio de Desv. Est.:" + str(MIN_Std))
  print("Desv. Est. del Promedio:" + str(MIN_Prom))
  print("\n")
  for i in range(x):
    print("GRUPO " + str(i+1))
    print(MEJORES_GRUPOS[i][["ID","Peso","Sexo"]])  
    print("\n")
  return(MEJORES_GRUPOS)

Adicional al algoritmo generado, en las siguientes celdas se hace una asignacion de cada uno de los grupos generados en los tratamientos que se desean. 

In [0]:
#FUNCION AUXILIAR PARA GENERAR COMBINACIONES
############################################

def comb_posibles(lista,num):
  return [i for i in combinations(lista,num)]

In [0]:
#ASIGNACION DE GRUPOS EN TRATAMIENTOS
#####################################

#grupos: grupos generados conforme el algoritmo de agrupamiento
#T: numero de tratamientos
def asignacion_tratamientos (grupos,T):
  
  x = len(grupos)
  lista_x = [i for i in range(x)]
  num_grupos_t = x//T
  grupos_trat_final = []
  
  for tratamiento in range(T):
  
    print ("TRATAMIENTO " + str(tratamiento+1) + ":")
    print("\n")
    
    MIN_Std = 10e10
    MIN_Prom = 10e10
    GRUPO_TRAT = []
    COMB_TRAT = []
  
    comb_t = comb_posibles(lista_x,num_grupos_t)
  
    for combinacion in comb_t:
      grupos_trat = []
    
      for g in combinacion:
        grupo_tmp = grupos[g]
        grupos_trat.append(grupo_tmp)
  
      CASO_Std, CASO_Prom = func_objetivo(grupos_trat,num_grupos_t) 
  
      if (CASO_Std < MIN_Std) and (CASO_Prom < MIN_Prom):
        MIN_Std = CASO_Std
        MIN_Prom = CASO_Prom
        GRUPO_TRAT = grupos_trat
        COMB_TRAT = combinacion
    
    for i in range(len(COMB_TRAT)): 
      lista_x.remove(COMB_TRAT[i])

    print("Promedio de Desv. Est.:" + str(MIN_Std))
    print("Desv. Est. del Promedio:" + str(MIN_Prom))
    print("\n")
    for i in range(num_grupos_t):
      print("GRUPO " + str(i+1))
      print(GRUPO_TRAT[i][["ID","Peso","Sexo"]])  
      print("\n")
  
      

__Calcula la complejidad del algoritmo por fuerza bruta__

Respuesta:

Dado que el algoritmo por fuerza bruta debe pasar por todas las posibles particiones y generar todas las combinaciones posibles, entonces la complejidad del algoritmo es factorial $O(N!)$. Esto puede ser notado facilmente al ver que en el algoritmo expuesto se hace un llamado a la funcion auxiliar *particiones_x* en la que se generan $\frac {1}{2} {N \choose {{\lfloor N/x \rfloor}}}$ posibilidades.

__(\*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta__

Respuesta:

En vista de la complejidad del algoritmo de fuerza bruta, vamos a diseñar un algoritmo metaheuristico que evalue particiones candidatas en un numero controlado de iteraciones. Para esto, se creara una funcion auxiliar que genere aleatoriamente particiones de naturaleza homogenea. Para un mayor afinamiento del algoritmo, se realizara una validacion cruzada de potenciales soluciones, comparandolas entre ellas y seleccionando la mejor a fin de escapar de potenciales minimos locales. La asignacion en los grupos se mantienen por fuerza bruta debido a que las alternativas posibles son relativamente bajas y no afectan en forma significativa la eficiencia del procesamiento. 

Las razones para creer que esto mejora el algoritmo de fuerza bruta son las siguientes:
- En primer lugar, dada la complejidad expuesta en el algoritmo de fuerza bruta, es necesario tener un mayor control del numero de casos estudiados sin tener que pasar necesariamente por todas las posibilidades existentes.
- Justamente relacionado a lo anterior, y siguiendo con el enunciado, el desafio finalmente es alcanzar la mayor homogeneidad posible, y no necesariamente la maxima homogeneidad por lo que una solucion aproximada puede ser satisfactoria.

In [0]:
#FUNCION AUXILIAR PARA GENERAR PARTICIONES ALEATORIAS Y HOMOGENEAS
##################################################################

def particiones_x_random(S,x):
  
  #Inicializaciones
  particion = []
  S_restantes = S
  x_restantes = x
  
  #Generacion de particiones
  for i in range(x):
    tam = len(S_restantes)//x_restantes
    elem = random.sample(S_restantes,tam)
    particion.append(elem)
    S_restantes = list(set(S_restantes).difference(elem)) 
    x_restantes = x_restantes - 1
  
  return particion

In [0]:
#ALGORITMO METAHEURISTICO
#########################

#df_terneros: dataframe con los datos a analizar
#x: numero de grupos
#iteraciones: número de iteraciones a ejecutar
#cv_num: numero de casos en los que realizar validacion cruzada.
def agrupamiento_metaheuristico(df_terneros,x,iteraciones,cv_num):
  
  #Inicializaciones para comparar en validacion cruzada
  MIN_Std_CV = 10e10 
  MIN_Prom_CV = 10e10
  MEJORES_GRUPOS_CV = []
  
  for cv in range(cv_num):
  
    #Inicializaciones de iteraciones
    MIN_Std = 10e10 
    MIN_Prom = 10e10
    MEJORES_GRUPOS = []
  
    #Evaluacion de funcion objetivo en cada iteracion
    for iteracion in range(iteraciones):
    
      particion = particiones_x_random(list(df_terneros["ID"]),x)
    
      df_part = []
    
      for elem in particion:
        df_tmp = df_terneros[df_terneros["ID"].isin(elem)]
        df_part.append(df_tmp)
    
      CASO_Std, CASO_Prom = func_objetivo(df_part,x) 
  
      if (CASO_Std < MIN_Std) and (CASO_Prom < MIN_Prom):
        MIN_Std = CASO_Std
        MIN_Prom = CASO_Prom
        MEJORES_GRUPOS = df_part
     
    if (MIN_Std < MIN_Std_CV) and (MIN_Prom < MIN_Prom_CV):
      MIN_Std_CV = MIN_Std
      MIN_Prom_CV = MIN_Prom
      MEJORES_GRUPOS_CV = MEJORES_GRUPOS

  #Resultados obtenidos
  print("Promedio de Desv. Est.:" + str(MIN_Std_CV))
  print("Desv. Est. del Promedio:" + str(MIN_Prom_CV))
  print("\n")
  for i in range(x):
    print("GRUPO " + str(i+1))
    print(MEJORES_GRUPOS_CV[i][["ID","Peso","Sexo"]])  
    print("\n")
  return(MEJORES_GRUPOS_CV)

__(\*)Calcula la complejidad del algoritmo__

Respuesta:

Para evaluar la complejidad de ete algoritmo considerar que el tiempo de ejecucion del mismo no depende de $N$ como tal, sino del numero de validaciones cruzadas $C$, del total de iteraciones $I$ y de la cantidad de grupos que se desean crear $G=x$. En este sentido, se puede decir que el algoritmo tiene una complejidad lineal en $G$ dada por $O(C \times I \times G)$.

Esta complejidad mejorara lo alcanzado en fuerza bruta siempre que 

$$C \times I \times G < N!$$

Dado que hemos acotado $G=9$ y $N=200$ es posible decir que la complejidad del problema en particular mejorara siempre que

$$C \times I < \frac {200!}{9} $$

__Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios__<br>

Respuesta:

Para la construcción del data set de entrada tomamos la estructura definida líneas arriba

In [10]:
#GENERACION DE DATA FRAME ALEATORIO CON PESOS Y SEXO
####################################################

df_terneros = pd.DataFrame(dicc)
df_terneros.sample(5)

,ID,Peso,Sexo
98,99,40.304812,H
52,53,32.001547,H
12,13,31.214236,H
56,57,35.444779,M
141,142,35.377064,H


__Aplica el algoritmo al juego de datos generado__

Respuesta:

Para la ejecucion del algoritmo, se considera a los 9 grupos tal que $G=x = 9$ y se trabajaran con $1000$ iteraciones y $5$ validaciones cruzadas.

In [11]:
#APLICACION DE ALGORITMO CREADO
###############################

resp = agrupamiento_metaheuristico(df_terneros,9,1000,5)

Promedio de Desv. Est.:2.1279142098426322
Desv. Est. del Promedio:0.5155040584064476


GRUPO 1
      ID       Peso Sexo
4      5  36.243274    H
16    17  35.852170    H
21    22  37.143247    M
22    23  35.847132    H
27    28  37.675743    H
30    31  35.641729    M
38    39  36.589232    M
46    47  40.732984    M
51    52  33.746079    M
59    60  36.727211    M
63    64  34.176101    H
64    65  38.374624    H
76    77  37.732731    M
92    93  35.961241    H
105  106  36.225930    H
111  112  40.481781    H
118  119  37.481532    H
137  138  40.304569    M
146  147  38.732562    H
154  155  35.135519    H
156  157  35.702547    H
179  180  38.617408    M


GRUPO 2
      ID       Peso Sexo
5      6  34.184111    M
9     10  40.533068    H
36    37  34.727365    M
48    49  40.640616    M
52    53  32.001547    H
56    57  35.444779    M
73    74  39.394058    H
89    90  32.290565    H
103  104  32.265472    H
129  130  33.740970    M
130  131  40.587557    M
138  139  37.352401 

In [12]:
#ASINACION EN 3 TRATAMIENTO DESEADOS
####################################

asignacion_tratamientos(resp,3)

TRATAMIENTO 1:


Promedio de Desv. Est.:1.8364545219212436
Desv. Est. del Promedio:0.18226748720672425


GRUPO 1
      ID       Peso Sexo
7      8  34.312662    M
8      9  34.562341    H
11    12  37.251302    H
18    19  34.936365    H
20    21  38.770035    M
25    26  38.531616    H
28    29  37.357187    H
62    63  36.075718    H
77    78  34.105786    M
100  101  36.862835    H
102  103  36.048321    H
108  109  39.597690    M
113  114  35.267393    H
128  129  37.593919    M
136  137  38.080873    M
158  159  36.557507    H
159  160  36.489455    M
173  174  35.601284    H
176  177  36.961044    M
187  188  36.964279    M
191  192  34.922088    M
196  197  37.006830    H


GRUPO 2
      ID       Peso Sexo
0      1  40.249947    M
10    11  32.623252    M
15    16  39.178184    H
32    33  33.614371    M
37    38  35.200024    M
41    42  37.654871    H
44    45  37.843200    M
66    67  36.941800    M
74    75  39.038401    H
78    79  38.973286    M
80    81  37.186665    M
91

__Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo__

Respuesta:

Para la resolucion de este problema se han considerado las siguientes referencias:

- Numeros de Stirling de Segundo Orden: https://en.wikipedia.org/wiki/Stirling_numbers_of_the_second_kind
- Funcion para generacion de particiones: https://stackoverflow.com/questions/19368375/set-partitions-in-python
- Conceptos relacionados con particiones: https://faculty.math.illinois.edu/~hildebr/347.summer14/relations.pdf


__Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño__

Respuesta:

En primer lugar considerar que es posible incorporar dentro del analisis al sexo como una variable de agrupamiento. Por ejemplo, se podria considerar como un parametro de la homogeneidad intra o inter grupos.

Otro aspecto que resulta de interes en este caso seria cambiar la restriccion de homogeneidad por su opuesto de heterogeneidad. esto motivaria primero el cambio de sentido de la optimizacion a buscar maximizar laa funcion objetivo. Segundo, ampliaria en gran medida el expectro de posibles resultados forzando a generar mejoras en el algoritmo metaheuristico desarrollado. (implementacion de logicas de algoritmos geneticos y mutaciones en los grupos obtenidos aleatoriamente por ejemplo)

Otro aspecto interesante puede ser en la posibilidad de generar grupos sin considerar a toda la poblacion, descartando por ejemplo terneros cuya presencia en cualquier grupo apalanque una menor homogeneidad.

## Problema:

__3. Combinar cifras y operaciones__

- Disponemos de las 9 cifras del 1 al 9 (excluimos el cero) y de los 4 signos básicos de las
operaciones fundamentales: suma(+), resta(-), multiplicación(*) y división(/)
- Debemos combinarlos alternativamente sin repetir ninguno de ellos para obtener una
cantidad dada. Un ejemplo sería para obtener el 4:
$$4+2-6/3*1 = 4$$
- Debe analizarse el problema para encontrar todos los valores enteros posibles planteando las
siguientes cuestiones:
  - ¿Qué valor máximo y mínimo a priori se pueden obtener según las condiciones?
  - ¿Es posible encontrar todos los valores enteros posibles entre dicho mínimo y máximo ?

__(\*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones?__ <br>

Respuesta:

Sea $ \omega $ el total de posibles operaciones que se deben seleccionar para ubicarlas en igual cantidad de espacios de una expresion aritmetica, y sea $N$ la cantidad de posibles numeros de donde escoger para mezclarlos con las operaciones. 

Entonces, notemos primero que por la alternabilidad la cantidad de numeros que pueden ser usados bajo toda cirunstancia esta dado por $\omega + 1$.

A continuacion, notemos tambien que es posible generar $\omega^{\omega}$ posibles posiciones de las operaciones, y que para cada una de estas posiciones, se pueden escoger $N^{\omega+1}$ numeros. 

Por lo expuesto, sin ningun tipo de restricciones las posibles soluciones ($X$) estan dadas por

$$X=\omega^{\omega}N^{\omega+1}={(\omega N)^{\omega}}N$$

Sustituyendo los términos con los valores del enunciado se tiene que

$$ X=(4 \times 9)^{4} \times 9 = 15,116,544 $$

__¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones?:__ <br>

Respuesta:

La primera restriccion viene dada porque no es posible repetir ningun numero, ni tampoco ninguna  operacion, esto reduce el universo de posibles soluciones a

$$X=Permutaciones(\omega,1) \times Permutaciones(N,5) = {\omega!}{\frac{N!}{(N-\omega-1)!}} = {4!}\frac{9!}{4!} = 9! = 362,880$$

La segunda restriccion se refiere a que debemos garantizar que el resultado de la expresion sea un valor entero, lo cual reduce las posibles soluciones a

$$ X={4!} \times (9*8*7*6*4) \leq 290,304$$

Para entender de donde se obtiene esto consideremos el caso de seleccionar los numeros $a,b,c,d,e$ y supongamos un orden de operaciones que me de la expresion

$$ a+b*c-d/e$$

La mezcla de signos de operacion está dada por $4!$, el valor $a$ se escoge entre 9 posibilidades, el $b$ por su parte entre 8 ya que se descarta $a$, en este sentido $c$ se escoge entre 7, $d$ entre 6, pero $e$ solamente puede escogerse en el mejor de los casos entre los elementos del subconjunto $[1,2,3,4]$ siempre que $a,b,c,d \notin [1,2,3,4]$. Cualquier valor mayor que 4 entregaria al ser un divisor entregaria siempre un valor no entero.

__Modelo para el espacio de soluciones<br>
(\*) ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguméntalo)__<br>

Respuesta:

Dado que tenemos que seleccionar entre numeros entre el 1 y el 9, podemos construir un arreglo secuencial con un valor minimo de 1 y un maximo de 9 (inclusive). 

Asi tambien, podemos construir una lista de caracteres que contenga los simbolos de las expresiones que vamos a utilizar "+", "-", "\*", y "/".

Estos dos vectores pasarían como parametros a considerar en nuestros algoritmos que resuelvan el problema. 

En este contexto, y a fin de mejores el algoritmo de resolucion por fuerza bruta puede ser conveniente representar a estos datos dentro de un arbol de decision tal que los nodos correspondan a numeros resultado de ejecutar operaciones aritmeticas con los numero existentes hasta dicho nodo y las aristas a las operaciones. Esto permite tener una mejor visualizacion del problema como de las posibles soluciones. Tomemos por ejemplo el siguiente caso en el que se han seleccionado los siguientes 5 numeros: 6, 4, 7, 2, 1. Uno de los caminos vendria dado por la expresion $6+4*7/2-1 = 19$ y su representacion en el arbol seria 

$$ 6 \rightarrow + \rightarrow 10 \rightarrow * \rightarrow 34 \rightarrow / \rightarrow 20 \rightarrow -  \rightarrow 19$$

Si intercambiamos por ejemplo las operaciones de suma y resta la rama sería:

$$ 6 \rightarrow - \rightarrow -2 \rightarrow * \rightarrow -14 \rightarrow / \rightarrow -7 \rightarrow +  \rightarrow -6$$




In [13]:
#GENERACION DE ESTRUCTURAS DE DATOS
###################################

#Parametros iniciales
N = 9

#Lista con numeros entre 1 y 9
numeros = [i for i in range(1,N+1)]
print("LISTA DE NUMEROS:")
print(numeros)

#Operaciones aritmeticas
operaciones = ["+", "-", "*", "/"]
print("SIGNOS DE OPERACION:")
print(operaciones)

LISTA DE NUMEROS:
[1, 2, 3, 4, 5, 6, 7, 8, 9]
SIGNOS DE OPERACION:
['+', '-', '*', '/']


__Según el modelo para el espacio de soluciones__<br>
__(\*)¿Cual es la función objetivo?__<br>

Respuesta:

La funcion objetivo corresponde a aquella que me permita evaluar una expresion en especifico generada tomando al azar 5 numeros cualquiera sin repeticion y mezclando randomicamente los simbolos de operacion.

Ante esta funcion es posible evidenciar que *a priori* no es posible obtener resultados mayores a $77$ ni menores a $-69$. 

Para comprender esto notemos que en consideracion a las restricciones posibles, la subexpresion que mayor incidencia tiene en una expresion final viene dada por $9*8$ ya que la misma asegura la mayor multiplicacion posible; desde alli, las mayores subexpresiones con division, suma y resta posible seria $+7-2/1$, o bien, $+7/1-2$, las cuales al adicionarse a la primera nos da

$$9*8+7-2/1 = 9*8+7/1-2 = 77$$

Notemos que cualquier alteracion de esta expresión ya sea en signos o en numeros reducirá el resultado final.

De forma contraria la subexpresion que garantiza un resultado bajo esta dada por $1-9*8$, de alli la subexpresion de menor magnitud como resultado esta dada por $+4/2$, por lo que 

$$ 1-8*9+4/2 = -69$$ 

Cualquier otra combinacion entrega necesariamente un resultado mayor.

<br>
__(\*)¿Es un problema de maximización o minimización?__ <br>

Respuesta:

El problema planteado es un problema de busqueda de un resultado en especifico, por lo tanto no podemos hablar de maximizacion o minimizacion.

In [0]:
#GENERACION DE FUNCION OBJETIVO
###############################

def evaluacion(expresion):
  return eval(expresion)

__Diseña un algoritmo para resolver el problema por fuerza bruta__<br>
Respuesta:

Por fuerza bruta, implica que debemos recorrer todo el universo de posibles soluciones hasta encontrar la primera que permita alcanzar el resultado buscado.

Vale señalar que para el funcionamiento de este algoritmo se toma como supuesto siempre que se deben seleccionar 5 numeros y existen 4 operaciones.

In [0]:
#ALGORITMO DE FUERZA BRUTA
##########################

#numeros: lista de numeros que vamos a probar
#operaciones: operaciones elementales a utilizar
#resultado deseado
def expresion_fuerzabruta (numeros,operaciones,resultado):

  #Validacion inicial dadas las restricciones del problema
  if (resultado < -69) or (resultado > 77):
    print ("NO EXISTE EXPRESION POSIBLE PARA DICHO RESULTADO")
  else:
  
    #Inicializaciones_generales
    iteraciones = 0
    expresion = "10e10"
  
    #Busqueda de la expresion
    while evaluacion(expresion) != resultado:
      
      #Detener si iteraciones posibles alcanzan el limite establecido
      if iteraciones == 15116554:
        print ("LIMITE DE ITERACIONES ALCANZADO")
        break
      
      for n1 in numeros:
        for n2 in numeros:
          for n3 in numeros:
            for n4 in numeros:
              for n5 in numeros:
                num_muestra = [n1,n2,n3,n4,n5]
                for op1 in operaciones:
                  for op2 in operaciones:
                    for op3 in operaciones:
                      for op4 in operaciones:
                        op_muestra = [op1,op2,op3,op4]
    
                        if len(set(op_muestra)) == 4:
                          if len(set(num_muestra)) == 5:
                            expresion = str(n1) + op1 + str(n2) + op2 +str(n3) + op3 +str(n4) + op4 + str(n5)
                            iteraciones += 1
  
    print ("ITERACIONES VALIDAS REALIZADAS: " + str(iteraciones))
    if iteraciones <= 15116554:
      print ("EXPRESION HALLADA: " + expresion_nueva)

__Calcula la complejidad del algoritmo por fuerza bruta__

Respuesta:

El algoritmo por fuerza bruta recorre la lista de 9 números en 5 ocasiones lo cual habla de una complejidad $O(N^5)$. Adicionalmente, recorre la lista de 4 operaciones en 4 ocasiones lo cual permite asumir que el algoritmo tiene una complejidad dada por:

$$ O({N^5}{\omega^4}) = 256 \times O(N^5)$$

__(\*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta__

Respuesta:

Para la mejora del algoritmo inicialmente planteado, se genera un algoritmo de búsqueda aleatoria que genere 5 numeros al azar sin repeticion y mezcle aleatoriamente los signos de operacion. Si la evaluacion del algoritmo da el resultado deseado entonces el mismo se detiene y arrojael output alcanzado.

Este algoritmo mejora el algoritmo por fuerza bruta ya que:
- No requiere recorrer la totalidad de las posibilidades bajo ningun concepto.
- Finalmente, se garantiza alcanzar una solucion deseada sobre la base del concepto de "barajar cartas" hasta la solucion óptima antes que por seguir un orden de prueba específico, de cada numero y de cada signo. Esto resulta conveniente en vista que finalmente no se plantea una minimizacion sino una búsqueda.
- Sin pérdida de generalidad y relacionado a lo anterior, se hace uso del concepto de muestreo sin reemplazo. 

In [0]:
#ALGORITMO BUSQUEDA ALEATORIA
#############################

#numeros: lista de numeros que vamos a probar
#operaciones: operaciones elementales a utilizar
#resultado deseado
def expresion_busqaleatoria(numeros,operaciones,resultado):
  
  #Validacion inicial dadas las restricciones del problema
  if (resultado < -69) or (resultado > 77):
    print ("NO EXISTE EXPRESION POSIBLE PARA DICHO RESULTADO")
  else:
  
    #Inicializaciones globales
    expresiones_evaluadas =[]
    expresion_nueva = "10e10"
    iteraciones = 0
  
    #Evaluacion
    while evaluacion(expresion_nueva) != resultado:
    
      #Detener si iteraciones posibles alcanzan el limite establecido
      if iteraciones == 362880:
        print ("LIMITE DE ITERACIONES ALCANZADO")
        break
    
      #Generacion de numeros aleatorios y mezca de operaciones
      numeros_muestra = np.random.choice(numeros,len(operaciones)+1,replace = False)
      operaciones_mezcla = random.sample(operaciones,len(operaciones))
    
      #Contruccion de la expresion
      expresion_nueva = str(numeros_muestra[0])
      for i in range(len(operaciones)):
        expresion_nueva = expresion_nueva + operaciones_mezcla[i] + str(numeros_muestra[i+1])
      
      #Control de iteraciones
      if expresiones_evaluadas.count(expresion_nueva) == 0:
        iteraciones += 1
        expresiones_evaluadas.append(expresion_nueva)
        
    print ("ITERACIONES VALIDAS REALIZADAS: " + str(iteraciones))
    if iteraciones <= 362880:
      print ("EXPRESION HALLADA: " + expresion_nueva)
    
    return iteraciones, expresion_nueva

In [17]:
#Prueba funcionamiento algoritmo
resul = expresion_busqaleatoria(numeros,operaciones,0)

ITERACIONES VALIDAS REALIZADAS: 7
EXPRESION HALLADA: 4/2+5*1-7


__(\*)Calcula la complejidad del algoritmo__

Respuesta:

En este caso el algoritmo tiene una complejidad que depende de $N$ al momento de realizarel muestre con reemplazo, lo cual puede considerarse como una operacion lineal en $N$. Ademas depende de las iteraciones ($I$) que se ejecuten, en este constexto se puede hablar de una complejidad constante dada por 

$$ I \times O(N)$$

__Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios__<br>

Respuesta:

Supongamos un nuevo juego de datos pero ampliando el problema a $N=15$

In [18]:
#JUEGO DE DATOS NUEVO
#####################

numeros2 = [i+1 for i in range(15)]
numeros2

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

__Aplica el algoritmo al juego de datos generado__

Respuesta:

Los resultados se muestran a continuacion

In [19]:
#Aplicaciones de algoritmo en nuevo juego de datos
##################################################

resp2 = expresion_busqaleatoria(numeros2,operaciones,50)

ITERACIONES VALIDAS REALIZADAS: 650
EXPRESION HALLADA: 14-4+8*5/1


Notemos que en este caso, y solo con la perspectiva de evidenciar el funcionamiento, no se evalúa el universo de posibles soluciones que se pueden alcanzar con $N=15$. Por esta razon en el ejemplo se toma comoresultado deseado 50.

__Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo__

Respuesta:

No se emplearon referencias en este caso.

__Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño__

Respuesta:

Las variaciones mas evidentes al problema serian las siguientes:
- Ampliar el universo de posibles respuestas a numeros no enteros, esto conllevaria a mayores formas de combinar numeros. Ya que las divisiones no enteras serian consideradas.
- Tambien se puede ampliar el numero de operaciones matematicas a potenciacion y raiz. Esto generarian entre otros desafios ampliar las respuestas a numeros complejos, o bien a imponer restricciones solamente en los reales. Con esto, ademas el espectro de resultados posibles seria mucho mas amplio en el sentido que $9*8$ seria muy inferior a por ejemplo $8^9$, planteando asi un interesante desafio.

Vale mencionar que este problema se asemeja al famoso problema ingles del "Countdown problem" por lo que su utilizacion podria vincularse igualmente.

## Problema:

__2. Organizar los horarios de partidos de La Liga__

- Desde la La Liga de fútbol profesional se pretende organizar los horarios de los partidos
de liga de de cada jornada. Se conocen algunos datos que nos deben llevar a diseñar un
algoritmo que realice la asignación de los partidos a los horarios que __maximice la
audiencia__.

- Los horarios disponibles se conocen a priori y son los siguientes:

Dia|Horas
--|--
Viernes|20
Sábado|12,16,18,20
Domingo|12,16,18,20
Lunes|20

- En primer lugar se clasifican los equipos en tres categorías según el numero de
seguidores que tienen relación directa con la audiencia. Hay __4 equipos__ en la
__categoría A__, __10 equipos__ de __categoría B__ y __6 equipos__ de __categoría C__.

- Se conoce estadísticamente la audiencia que genera cada partido según los equipos
que se enfrentan y en horario de sábado a las 20h (el mejor en todos los casos).

Categoria|A|B|C
--|--|--|--
__A__|2 mill|1.3 mill|1 mill
__B__||0.9 mill|0.75 mill
__C__|||0.47 mill

- Si el horario del partido no se realiza a las 20 horas del sábado se sabe que se reduce
según los coeficientes de la siguiente tabla:

Hora/Dia|Viernes|Sabado|Domingo|Lunes
--|--|--|--|--
__12h__|-|0.55|0.45|-
__16h__|-|0.7|0.75|-
__18h__|-|0.8|0.85|-
__20h__|0.4|1|1|0.4

- Debemos asignar obligatoriamente siempre un partido el viernes y un partido el lunes.

-  


__(\*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones?__ <br>

Respuesta:

Sea $N$ los equipos que participan en el torneo, tal que el número de partidos por $F$ fechas es igual a $P = \lfloor N/2 \rfloor$. Para simplicidad se supondrá que $N$ siempre es un número par por lo que $P = N/2$.

Podemos saber que en cada fecha el número de posibiliades de partidos viene dado por:

$${\frac{1}{2}}{N \choose 2}{{N-2} \choose 2}{{N-4} \choose 2}...{{2} \choose 2} = {\frac{1}{2}}\prod^{P-1}_{i=0}{{{N-2i} \choose 2}} $$

Sabemos también que al asumir un torneo "todos contra todos" el número de fechas estará dado por $F=N-1$ por lo que el total de posibles soluciones $X$ sin restriccion es de

$$X= {\frac{N-1}{2}}\prod^{P-1}_{i=0}{{{N-2i} \choose 2}}$$

Sustituyendo los términos por loa valores dados en el enunciado, se obtiene que

$$X= {\frac{19}{2}}\prod^{9}_{i=0}{{{20-2i} \choose 2}} \approx 2.26 \times 10^{16}$$

__¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones?:__ <br>

Respuesta:

En el problema no se presentan restricciones que afecten el número de posibles soluciones. 

__Modelo para el espacio de soluciones<br>
(\*) ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguméntalo)__<br>

Respuesta:

Es posible construir un conjunto de datasets que nos guarden toda la informacion relacionada a los equipos, partidos, horarios y audiencias. Estos serán parametros al momento de construir nuestro algoritmo.

Dentro del algoritmo, podemos combinar estos datasets para así obtener también una tabla con el calendario final, y la audiencia esperada que se obtendría mediante cálculos vinculados a estos datasets.

El dataset de partidos inicialmente presenta el detalle de los dos equipos que se enfrentan y la fecha a la que corresponde el enfrentamiento.

El dataset de equipos inicialmente presente el identificador del equipo junto con la clase inicial (A, B o C) que se asigna arbitrariamente.

El dataset de horarios da cuenta del factor esperado en cada horario  respecto a la audicencia de "S20" (Sabado 20 horas).

Finalmente, el dataset de audiencias muestra la audiencia en millones para cada posible cruce de clases en el horario de "S20".

Vale mencionar que el desafio expuesto en el problema vendría dado por clasificar de mejor forma a los equipos conforme las categorías A, B y C, el resto de datasets estaría fijo en toda la resolución.

Esta estructura de datos resulta adecuada principalmente porque permite mantener una adecuada organizacion de la informacion requerida para la resolucion del problema, así también permite generar cruces entre tablas sin perder ningun tipo de informacion a lo largo de las posibles iteraciones que realice nuestro algoritmo.

In [0]:
#ESTRUCTURA DE DATOS
####################

#Parametros iniciales
N = 20

#ID de equipos inicial
ID_eq = [i+1 for i in range(N)]
ID_eq_rev = [N-i for i in range(N)]

#Matriz de partidos a generar
eq_izq = ID_eq[:10]
eq_der = ID_eq_rev[:10]

matriz_final = np.column_stack((eq_izq, eq_der,[1 for i in range(len(eq_izq))]))

IZQ = ID_eq
DER = ID_eq_rev

for fecha in range(2,N):
  
  if fecha <= 10:
    eq_izq = [IZQ[0]] + IZQ[N+1-fecha:] + IZQ[1:10-fecha+1] 
    eq_der = DER[fecha-1:9+fecha]
  else:
    eq_izq = [IZQ[0]] + IZQ[N+1-fecha:N+10-fecha]
    eq_der = DER[fecha-1:N-1] + DER[:fecha-10]
  
  mat_fecha = np.column_stack((eq_izq, eq_der,[fecha for i in range(len(eq_izq))]))

  matriz_final = np.vstack([matriz_final,mat_fecha])

#DataFrame con partidos
col_nombres = ["EQ1","EQ2","Fecha"]
partidos = pd.DataFrame(matriz_final,columns = col_nombres)
#partidos

#ID de equipos final
ID_eq = [i+1 for i in range(N)]

#Clasificacion inicial de equipos
clase_eq = ["A" for i in range(4)] + ["B" for i in range(10)] + ["C" for i in range(6)] 
#clase_eq

#DataFrame con equipos y clasificacion inicial
col_nombres = ["EQ","Clase"]
equipos = pd.DataFrame(np.column_stack((ID_eq,clase_eq)),columns = col_nombres)
for i in range(len(equipos)):
  equipos["EQ"][i] = int(equipos["EQ"][i])
#equipos

#DataFrame de horarios
horarios_opc = ["V20", "S12", "S16", "S18", "S20", "D12", "D16", "D18", "D20", "L20"]
factores = [0.4,0.55,0.7,0.8,1,0.45,0.75,0.85,1,0.4]
col_nombres = ["Horario","Factor"]

horarios = pd.DataFrame(np.column_stack((horarios_opc,factores)),columns = col_nombres)
for i in range(len(horarios)):
  horarios["Factor"][i] = float(horarios["Factor"][i])
#horarios

#Dataframe de audiencias
tipo_partido = ["AA","AB","BA","AC","CA","BB","BC","CB","CC"]
aud = [2,1.3,1.3,1,1,0.9,0.75,0.75,0.47]
col_nombres = ["Tipo_Partido","Audiencia"]

audiencia = pd.DataFrame(np.column_stack((tipo_partido,aud)),columns = col_nombres)
for i in range(len(audiencia)):
  audiencia["Audiencia"][i] = float(audiencia["Audiencia"][i])
#audiencia

__Según el modelo para el espacio de soluciones__<br>
__(\*)¿Cual es la función objetivo?__<br>

Respuesta:

La función objetivo corresponde a la suma total de la audicencia que se genere de los 190 partidos disputados en el torneo.


<br>
__(\*)¿Es un problema de maximización o minimización?__ <br>

Respuesta:

En vista del enunciado, el problema corresponde a uno de maximización.


In [0]:
#FUNCION OBJETIVO
#################

#calendario: data frame que se genere deñ desarrollo del algoritmo
def audiencia_total (calendario):
  return sum(calendario["Aud_efectiva"])

__Diseña un algoritmo para resolver el problema por fuerza bruta__<br>
Respuesta:



__Calcula la complejidad del algoritmo por fuerza bruta__

Respuesta:


__(\*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta__

Respuesta:

El siguiente algoritmo se emplea una logica de evolucion y mutacion para lo cual se crean funciones auxiliares que permiten generar modificaciones en las clases de los equipos.

In [22]:
#GENERACION DE CALENDARIO DE INICIO
###################################

#Generacion aleatoria de horarios
horarios_aleatorios = random.sample(horarios_opc,len(horarios_opc))

for fecha in range(2,N):
  horarios_aleatorios = horarios_aleatorios + random.sample(horarios_opc,len(horarios_opc))

#Combinacion de datasets
calendario = partidos
calendario["Horario"] = horarios_aleatorios

calendario = calendario.merge(horarios, on = "Horario")
calendario = calendario.merge(equipos, left_on = "EQ1", right_on = "EQ")
calendario = calendario.drop(["EQ"], axis = 1)
calendario = calendario.merge(equipos, left_on = "EQ2", right_on = "EQ", suffixes = ["_1","_2"])
calendario = calendario.drop(["EQ"], axis = 1)
calendario["Tipo_Partido"] = calendario.apply(lambda x: x["Clase_1"] + x["Clase_2"],axis=1)
calendario = calendario.merge(audiencia,on = "Tipo_Partido")
calendario["Aud_efectiva"] = calendario.apply(lambda x: x["Factor"] * x["Audiencia"],axis=1)

#Presentacion de calendario inicial
calendario = calendario.sort_values(["Fecha"])
calendario.sample(10)

,EQ1,EQ2,Fecha,Horario,Factor,Clase_1,Clase_2,Tipo_Partido,Audiencia,Aud_efectiva
66,4,13,3,D18,0.85,A,B,AB,1.3,1.1050
76,4,9,5,S20,1,A,B,AB,1.3,1.3000
119,5,14,2,L20,0.4,B,B,BB,0.9,0.3600
108,16,12,7,S18,0.8,C,B,CB,0.75,0.6000
68,3,10,5,D12,0.45,A,B,AB,1.3,0.5850
54,5,18,19,D12,0.45,B,C,BC,0.75,0.3375
168,6,4,16,S12,0.55,B,A,BA,1.3,0.7150
15,15,20,13,S18,0.8,C,C,CC,0.47,0.3760
111,19,11,6,D18,0.85,C,B,CB,0.75,0.6375
33,9,20,16,D16,0.75,B,C,BC,0.75,0.5625


In [0]:
#FUNCION AUXILIAR PARA AJUSTE DE CLASES POR EVOLUCION
#####################################################

def evol_clases (calendario,equipos):
  
  ID_eq = [equipo for equipo in equipos["EQ"]]
  aud = []
  
  for e in ID_eq:
    mask = (calendario["EQ1"] == e) | (calendario["EQ2"] == e) 
    calendario_subset = calendario[mask]
    aud.append(sum(calendario_subset["Aud_efectiva"]))
  
  #Evolucion de clases de equipos
  equipos_aud = equipos
  equipos_aud["Aud"] = aud
  equipos_aud = equipos_aud.sort_values(["Aud"], ascending = False).reset_index()
  equipos_aud = equipos_aud.drop(["index"],axis = 1)
  
  equipos_A = equipos_aud[equipos_aud["Clase"] == "A"].reset_index()
  equipos_A = equipos_A.drop(["index"],axis = 1)
  equipos_B = equipos_aud[equipos_aud["Clase"] == "B"].reset_index()
  equipos_B = equipos_B.drop(["index"],axis = 1)
  equipos_C = equipos_aud[equipos_aud["Clase"] == "C"].reset_index()
  equipos_C = equipos_C.drop(["index"],axis = 1)
  
  equipos_A.loc[[3],["Clase"]] = "B"
  equipos_B.loc[[0],["Clase"]] = "A"
  equipos_B.loc[[9],["Clase"]] = "C"
  equipos_C.loc[[0],["Clase"]] = "B"
  
  equipos_new = equipos_A.append(equipos_B)
  equipos_new = equipos_new.append(equipos_C)
  equipos_new = equipos_new.drop(["Aud"],axis = 1)
  equipos_new = equipos_new.sort_values(["EQ"]).reset_index()
  equipos_new = equipos_new.drop(["index"],axis = 1)
  
  return equipos_new  

In [0]:
 #AJUSTE DE CLASES POR MUTACION NO CONTROLADA
 ############################################

def mut_clases (equipos,pmut):
  
  equipos_new = equipos
  ID_eq = [equipo for equipo in equipos["EQ"]]
  
  if random.uniform(0,1) < pmut:
    #print("MUT")
    casos = random.sample(ID_eq,2)
    mut1 = equipos_new.loc[casos[1]-1]["Clase"]
    mut2 = equipos_new.loc[casos[0]-1]["Clase"]
    if mut1 != mut2:
      equipos_new.loc[[casos[0]-1],["Clase"]]= mut1
      equipos_new.loc[[casos[1]-1],["Clase"]] = mut2
  
  return equipos_new

In [0]:
#ALGORITMO EVOLUTIVO
####################

#calendario_inicial: es el calendario inicial creado lineas arriba
#equipos,partidos,horarios_aleatorios,horarios,audiencis: son los data set creados en la estructuracion de los datos
#pmut: es una probabilidad de mutacion definida
#umbral_aud: es el umbral de audiencia minimo que se desea alcanzar
# iteraciones: son las iteraionces que deseamos correr para alcanzar un maximo
def mejor_calendario(calendario_inicial,equipos,partidos,horarios_aleatorios,horarios,audiencia,pmut,umbral_aud,iteraciones):
  
  MAX_Aud = umbral_aud
  calendario = calendario_inicial
  N = len(equipos)
  
  for i in range(iteraciones):
    
    Aud_caso = audiencia_total(calendario)
    
    if i <= iteraciones - 1:
      if Aud_caso > MAX_Aud:
        MAX_Aud = Aud_caso
        equipos = mut_clases(equipos,pmut)
      else:
        equipos = evol_clases(calendario,equipos)

    #Combinacion de datasets
    calendario = partidos
    calendario["Horario"] = horarios_aleatorios

    calendario = calendario.merge(horarios, on = "Horario")
    calendario = calendario.merge(equipos, left_on = "EQ1", right_on = "EQ")
    calendario = calendario.drop(["EQ"], axis = 1)
    calendario = calendario.merge(equipos, left_on = "EQ2", right_on = "EQ", suffixes = ["_1","_2"])
    calendario = calendario.drop(["EQ"], axis = 1)
    calendario["Tipo_Partido"] = calendario.apply(lambda x: x["Clase_1"] + x["Clase_2"],axis=1)
    calendario = calendario.merge(audiencia,on = "Tipo_Partido")
    calendario["Aud_efectiva"] = calendario.apply(lambda x: x["Factor"] * x["Audiencia"],axis=1)
    calendario = calendario.sort_values(["Fecha"]) 
  
  print("AUDIENCIA MAXIMA LOGRADA: " + str(MAX_Aud) + "mill.")
  print("\n")
  print("CATEGORIAS DE EQUIPOS:")
  print(equipos)
  print("\n")
  print("CALENDARIO:")
  print(calendario)

In [26]:
#Evaluacion de la funcion
#########################

mejor_calendario(calendario,equipos,partidos,horarios_aleatorios,horarios,audiencia,0.1,100,500)

AUDIENCIA MAXIMA LOGRADA: 127.8595mill.


CATEGORIAS DE EQUIPOS:
    EQ Clase
0    1     B
1    2     B
2    3     B
3    4     A
4    5     C
5    6     B
6    7     C
7    8     C
8    9     B
9   10     A
10  11     C
11  12     A
12  13     B
13  14     C
14  15     B
15  16     C
16  17     B
17  18     B
18  19     A
19  20     B


CALENDARIO:
    EQ1 EQ2  Fecha Horario Factor Clase_1 Clase_2 Tipo_Partido Audiencia  \
0     1  20      1     V20    0.4       B       B           BB       0.9   
152   2  19      1     D16   0.75       B       A           BA       1.3   
31    3  18      1     S16    0.7       B       B           BB       0.9   
158   9  12      1     L20    0.4       B       A           BA       1.3   
132   5  16      1     S20      1       C       C           CC      0.47   
18    6  15      1     D18   0.85       B       B           BB       0.9   
52    8  13      1     S12   0.55       C       B           CB      0.75   
81    4  17      1     S18    0.8       

__(\*)Calcula la complejidad del algoritmo__

Respuesta:

Este algoritmo presenta una complejidad cuadratica dada la existencia de lazos anidados en la funcion principal y en las auxiliares.

__Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios__<br>

Respuesta:


__Aplica el algoritmo al juego de datos generado__

Respuesta:


__Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo__

Respuesta:


__Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño__

Respuesta:
